<a href="https://colab.research.google.com/github/reloef/Jobs_Recommender/blob/master/Copy_of_Data_collection_and_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vacancy Dashboard data from CareerJet

In [1]:

#Import warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

# imports for NLP
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.pipeline import Pipeline
import pickle


# feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocessing
#import contractions
import os
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from textblob import TextBlob
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

##Extracting data from CareerJet

In [2]:
from google.colab import files
uploaded = files.upload()

Saving SA.csv to SA.csv


In [3]:
import io

In [4]:
df = pd.read_csv(io.BytesIO(uploaded['SA.csv']))
print(df)

                     locations  site                       date  \
0        Johannesburg, Gauteng   NaN  2022-04-29 07:52:13+00:00   
1            Pretoria, Gauteng   NaN  2022-04-24 07:36:21+00:00   
2        Johannesburg, Gauteng   NaN  2022-04-20 23:49:44+00:00   
3        Midrand, Johannesburg   NaN  2022-04-15 06:38:12+00:00   
4        Midrand, Johannesburg   NaN  2022-04-15 01:04:44+00:00   
5                      Gauteng   NaN  2022-04-14 22:52:30+00:00   
6            Pretoria, Gauteng   NaN  2022-04-13 22:27:41+00:00   
7                      Gauteng   NaN  2022-04-08 22:39:14+00:00   
8        Johannesburg, Gauteng   NaN  2022-04-08 07:24:27+00:00   
9      Cape Town, Western Cape   NaN  2022-04-07 22:21:43+00:00   
10       Johannesburg, Gauteng   NaN  2022-04-07 22:09:51+00:00   
11       Midrand, Johannesburg   NaN  2022-04-05 23:50:38+00:00   
12          Centurion, Gauteng   NaN  2022-04-05 22:15:14+00:00   
13          Centurion, Gauteng   NaN  2022-04-04 22:22:02+00:0

In [5]:
#Convert SA['jobs'] into a DataFrame
df_Jobs = pd.DataFrame.from_dict(df)

In [6]:
#Display the Dataframe
df_Jobs.head()

,locations,site,date,url,title,description,company,salary,salary_min,salary_type,salary_currency_code,salary_max
0,"Johannesburg, Gauteng",NaN,2022-04-29 07:52:13+00:00,http://jobviewtrack.com/en-za/job-1f1841694c10...,Data Analytics Sales Specialist,"Warehouses, <b>Data</b> Streaming, ETL, <b>Da...",Google,NaN,NaN,NaN,NaN,NaN
1,"Pretoria, Gauteng",NaN,2022-04-24 07:36:21+00:00,http://jobviewtrack.com/en-za/job-4f13416c7a37...,AWS Data Science Architect – Semi Remote – R74...,AWS <b>Data</b> <b>Science</b> Architect to j...,e-Merge IT Recruitment,R619 - 742 per hour,619.0,H,ZAR,742.0
2,"Johannesburg, Gauteng",NaN,2022-04-20 23:49:44+00:00,http://jobviewtrack.com/en-za/job-121341694c10...,"Consultant, Data Science and Analytics","Consultant, <b>Data</b> <b>Science</b> and ...",TransUnion,NaN,NaN,NaN,NaN,NaN
3,"Midrand, Johannesburg",NaN,2022-04-15 06:38:12+00:00,http://jobviewtrack.com/en-za/job-1c1b41694c10...,AWS Data Science Architect (Contract) Midrand/...,AWS <b>Data</b> <b>Science</b> Architect (Cont...,iSanqa Resourcing,NaN,NaN,NaN,NaN,NaN
4,"Midrand, Johannesburg",NaN,2022-04-15 01:04:44+00:00,http://jobviewtrack.com/en-za/job-1c1b41694c10...,AWS Data Science Architect (Contract) Midrand/...,AWS <b>Data</b> <b>Science</b> Architect (Cont...,iSanqa Resourcing,NaN,NaN,NaN,NaN,NaN


In [7]:
#View the shape of df_Jobs
df_Jobs.shape

(99, 12)

In [8]:
#View the info of df_Jobs
df_Jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   locations             99 non-null     object 
 1   site                  0 non-null      float64
 2   date                  99 non-null     object 
 3   url                   99 non-null     object 
 4   title                 99 non-null     object 
 5   description           99 non-null     object 
 6   company               83 non-null     object 
 7   salary                13 non-null     object 
 8   salary_min            13 non-null     float64
 9   salary_type           13 non-null     object 
 10  salary_currency_code  13 non-null     object 
 11  salary_max            13 non-null     float64
dtypes: float64(3), object(9)
memory usage: 9.4+ KB


In [9]:
# Check for the sum and percentage of null values in each column
total_nans = df_Jobs.isnull().sum().sort_values(ascending=False)
percent_nans = (df_Jobs.isnull().mean() * 100).sort_values(ascending=False)
df_clean_nans = pd.concat([total_nans, percent_nans], axis=1, keys=['Total NaN', 'Percent NaN'])
df_clean_nans.head()

,Total NaN,Percent NaN
site,99,100.000000
salary,86,86.868687
salary_min,86,86.868687
salary_type,86,86.868687
salary_currency_code,86,86.868687


In [10]:
#Make a copy of the DataFrame
df_new = df_Jobs.copy()

In [11]:
#Delete column that won't be useful
del df_new['site']
del df_new['url']
del df_new['salary_min']
del df_new['salary_type']
del df_new['salary_currency_code']
del df_new['salary_max']

In [12]:
#Read all text in a column
pd.set_option("display.max_colwidth", -1)

In [13]:
#display the text in df_Jobs['description']
df_new['description'][2]

'   Consultant, <b>Data</b> <b>Science</b> and Analytics... research techniques and software and presents findings to peers within TransUnion. Manages the integration of <b>data</b> analysis...'

In [14]:
df_new.head()

,locations,date,title,description,company,salary
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,"Warehouses, <b>Data</b> Streaming, ETL, <b>Data</b> governance, Business Intelligence, Predictive Analytics, OLAP, <b>Data</b> <b>science</b> framework... experience. Experience in selling <b>data</b> analytics software or solutions. Experience with the global analytics market...",Google,NaN
1,"Pretoria, Gauteng",2022-04-24 07:36:21+00:00,AWS Data Science Architect – Semi Remote – R742 Per hour,"AWS <b>Data</b> <b>Science</b> Architect to join their dynamic team. In this role you will be responsible for Migration of services... with: General AWS experience in the <b>Data</b> <b>Science</b> / Big <b>Data</b> context. Docker, & Kubernetes Experience, Expert JAVA developer...",e-Merge IT Recruitment,R619 - 742 per hour
2,"Johannesburg, Gauteng",2022-04-20 23:49:44+00:00,"Consultant, Data Science and Analytics","Consultant, <b>Data</b> <b>Science</b> and Analytics... research techniques and software and presents findings to peers within TransUnion. Manages the integration of <b>data</b> analysis...",TransUnion,NaN
3,"Midrand, Johannesburg",2022-04-15 06:38:12+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102000,AWS <b>Data</b> <b>Science</b> Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102000 About the job AWS <b>Data</b>... COMMENCEMENT: As soon as possible ESSENTIAL REQUIREMENTS: General AWS experience in the <b>Data</b> <b>Science</b> / Big <b>Data</b> context...,iSanqa Resourcing,NaN
4,"Midrand, Johannesburg",2022-04-15 01:04:44+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102004,AWS <b>Data</b> <b>Science</b> Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102004 About the job AWS <b>Data</b>... <b>Science</b> Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102004 Our client requires the services...,iSanqa Resourcing,NaN


In [15]:
df_new['description'].head()


0     Warehouses, <b>Data</b> Streaming, ETL, <b>Data</b> governance, Business Intelligence, Predictive Analytics, OLAP, <b>Data</b> <b>science</b> framework... experience.  Experience in selling <b>data</b> analytics software or solutions.  Experience with the global analytics market...          
1     AWS <b>Data</b> <b>Science</b> Architect to join their dynamic team.   In this role you will be responsible for Migration of services... with:  General AWS experience in the <b>Data</b> <b>Science</b> / Big <b>Data</b> context.  Docker, & Kubernetes Experience,  Expert JAVA developer...     
2       Consultant, <b>Data</b> <b>Science</b> and Analytics... research techniques and software and presents findings to peers within TransUnion. Manages the integration of <b>data</b> analysis...                                                                                                     
3    AWS <b>Data</b> <b>Science</b> Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - 

#Data Cleaning

In [16]:
#Define char and print punctuation
char = ['<b>','</b>']
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [17]:
from textblob import Word
from nltk.corpus import stopwords
import unicodedata

#Remove char from df_jobs['description']
df_new['description'] = df_new['description'].replace(to_replace = char, value="" , regex = True)

#Function to remove punctuation
def remove_punctuation(txt):
    return ''.join([l for l in txt if l not in string.punctuation])
df_new['description'] = df_new['description'].apply(remove_punctuation)

#Convert to lowercase
df_new['description'] = df_new['description'].str.lower()

#remove stop words
nltk.download('stopwords')
#Download wordnet
nltk.download('wordnet')
ADDITIONAL_STOPWORDS = ['covfefe']
stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
df_new['description'] = df_new['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

## lemmatization
df_new['description'] = df_new['description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# Remove extra whitespace
#df_jobs['description'] = re.sub(r'\s\s+', ' ', str(df_jobs['description'])) ##this line of code is an issue
#(It is the one that leads to the repetition of row 1)

df_new.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,locations,date,title,description,company,salary
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN
1,"Pretoria, Gauteng",2022-04-24 07:36:21+00:00,AWS Data Science Architect – Semi Remote – R742 Per hour,aws data science architect join dynamic team role responsible migration service general aws experience data science big data context docker kubernetes experience expert java developer,e-Merge IT Recruitment,R619 - 742 per hour
2,"Johannesburg, Gauteng",2022-04-20 23:49:44+00:00,"Consultant, Data Science and Analytics",consultant data science analytics research technique software present finding peer within transunion manages integration data analysis,TransUnion,NaN
3,"Midrand, Johannesburg",2022-04-15 06:38:12+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102000,aws data science architect contract midrandmenlynrosslynhome office rotation isb2102000 job aws data commencement soon possible essential requirement general aws experience data science big data context,iSanqa Resourcing,NaN
4,"Midrand, Johannesburg",2022-04-15 01:04:44+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102004,aws data science architect contract midrandmenlynrosslynhome office rotation isb2102004 job aws data science architect contract midrandmenlynrosslynhome office rotation isb2102004 client requires service,iSanqa Resourcing,NaN


Noticed that the ngrams don't include integers hence the conversion of 'int' into 'str'

In [18]:
#Convert int into str
#df_new['description'] = df_new['description'].astype(str)


In [19]:
#df_new.dtypes

In [20]:
df_new['description'][0]

'warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market'

In [21]:
df_new.head()

,locations,date,title,description,company,salary
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN
1,"Pretoria, Gauteng",2022-04-24 07:36:21+00:00,AWS Data Science Architect – Semi Remote – R742 Per hour,aws data science architect join dynamic team role responsible migration service general aws experience data science big data context docker kubernetes experience expert java developer,e-Merge IT Recruitment,R619 - 742 per hour
2,"Johannesburg, Gauteng",2022-04-20 23:49:44+00:00,"Consultant, Data Science and Analytics",consultant data science analytics research technique software present finding peer within transunion manages integration data analysis,TransUnion,NaN
3,"Midrand, Johannesburg",2022-04-15 06:38:12+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102000,aws data science architect contract midrandmenlynrosslynhome office rotation isb2102000 job aws data commencement soon possible essential requirement general aws experience data science big data context,iSanqa Resourcing,NaN
4,"Midrand, Johannesburg",2022-04-15 01:04:44+00:00,AWS Data Science Architect (Contract) Midrand/Menlyn/Rosslyn/Home Office rotation - ISB2102004,aws data science architect contract midrandmenlynrosslynhome office rotation isb2102004 job aws data science architect contract midrandmenlynrosslynhome office rotation isb2102004 client requires service,iSanqa Resourcing,NaN


In [22]:
#Create new columns in df_new

#Experience column
df_new['experience'] = np.nan

#Skills column
df_new['skills'] = np.nan

#Qualification column
df_new['qualification'] = np.nan

In [23]:
#Create columns for ngrams
from nltk.util import ngrams

n = 1
df_new['one_grams'] = df_new['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_new.assign(count=df_new['one_grams'].str.len())
   .explode('one_grams')
   #.query('count > 0')
)

n = 2
df_new['two_grams'] = df_new['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_new.assign(count=df_new['two_grams'].str.len())
   .explode('two_grams')
   .query('count > 0')
)

n = 3
df_new['three_grams'] = df_new['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_new.assign(count=df_new['three_grams'].str.len())
   .explode('three_grams')
   .query('count > 0')
)

df_new.head()

,locations,date,title,description,company,salary,experience,skills,qualification,one_grams,two_grams,three_grams
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN,NaN,NaN,NaN,"[warehouse, data, streaming, etl, data, governance, business, intelligence, predictive, analytics, olap, data, science, framework, experience, experience, selling, data, analytics, software, solution, experience, global, analytics, market]","[warehouse data, data streaming, streaming etl, etl data, data governance, governance business, business intelligence, intelligence predictive, predictive analytics, analytics olap, olap data, data science, science framework, framework experience, experience experience, experience selling, selling data, data analytics, analytics software, software solution, solution experience, experience global, global analytics, analytics market]","[warehouse data streaming, data streaming etl, streaming etl data, etl data governance, data governance business, governance business intelligence, business intelligence predictive, intelligence predictive analytics, predictive analytics olap, analytics olap data, olap data science, data science framework, science framework experience, framework experience experience, experience experience selling, experience selling data, selling data analytics, data analytics software, analytics software solution, software solution experience, solution experience global, experience global analytics, global analytics market]"
1,"Pretoria, Gauteng",2022-04-24 07:36:21+00:00,AWS Data Science Architect – Semi Remote – R742 Per hour,aws data science architect join dynamic team role responsible migration service general aws experience data science big data context docker kubernetes experience expert java developer,e-Merge IT Recruitment,R619 - 742 per hour,NaN,NaN,NaN,"[aws, data, science, architect, join, dynamic, team, role, responsible, migration, service, general, aws, experience, data, science, big, data, context, docker, kubernetes, experience, expert, java, developer]","[aws data, data science, science architect, architect join, join dynamic, dynamic team, team role, role responsible, responsible migration, migration service, service general, general aws, aws experience, experience data, data science, science big, big data, data context, context docker, docker kubernetes, kubernetes experience, experience expert, expert java, java developer]","[aws data science, data science architect, science architect join, architect join dynamic, join dynamic team, dynamic team role, team role responsible, role responsible migration, responsible migration service, migration service general, service general aws, general aws experience, aws experience data, experience data science, data science big, science big data, big data context, data context docker, context docker kubernetes, docker kubernetes experience, kubernetes experience expert, experience expert java, expert java developer]"
2,"Johannesburg, Gauteng",2022-04-20 23:49:44+00:00,"Consultant, Data Science and Analytics",consultant data science analytics research technique software present finding peer within transunion manages integration data analysis,TransUnion,NaN,NaN,NaN,NaN,"[consultant, data, science, analytics, research, technique, software, present, finding, peer, within, transunion, manages, integration, data, analysis]","[consultant data, data science, science analytics, analytics research, research technique, technique software, software present, present finding, finding peer, peer within, within transunion, transunion manages, manages integration, integration data, data analysis]","[consultant data science, data science analytics, science analytics research, analytics research technique, rese

In [24]:
#df_new =pd.Dataframe({'keys':['python','sql','aws','analyst','powerbi','statistics','pyspark','github','developer','programming','database','scrum','coding','visualization','docker']})

#df_new['2gramsxx'] = df_new['one_grams'].apply(lambda x: skil_dict.get(x))
#df_new['2gramsxx'] = df_new['one_grams'].map(skil_dict)

#df_new['match1'] = [dict_2 for _ in range(len(df_new))]
#df_new['col5'] = [dict_2]*len(df_new)

df_new.head(1)

#df = pd.DataFrame({'Title':['small test','huge Test', 'big','nothing', np.nan, 'a', 'b']})


,locations,date,title,description,company,salary,experience,skills,qualification,one_grams,two_grams,three_grams
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN,NaN,NaN,NaN,"[warehouse, data, streaming, etl, data, governance, business, intelligence, predictive, analytics, olap, data, science, framework, experience, experience, selling, data, analytics, software, solution, experience, global, analytics, market]","[warehouse data, data streaming, streaming etl, etl data, data governance, governance business, business intelligence, intelligence predictive, predictive analytics, analytics olap, olap data, data science, science framework, framework experience, experience experience, experience selling, selling data, data analytics, analytics software, software solution, solution experience, experience global, global analytics, analytics market]","[warehouse data streaming, data streaming etl, streaming etl data, etl data governance, data governance business, governance business intelligence, business intelligence predictive, intelligence predictive analytics, predictive analytics olap, analytics olap data, olap data science, data science framework, science framework experience, framework experience experience, experience experience selling, experience selling data, selling data analytics, data analytics software, analytics software solution, software solution experience, solution experience global, experience global analytics, global analytics market]"


In [39]:
words  = ['python','sql','java','aws','analyst','architect','powerbi','statistics','pyspark','github','excel','developer','programming','database','scrum','coding','visualization','docker','software','kubernetes']
words2 = ['machine learning','data analyst','data analytics','model development','power bi','cloud computing','data science','ms azure','business intelligence','statistical modelling','predictive analytics','microsoft excel','big data']

In [47]:

df_new['match_words'] = df_new['one_grams'].apply(lambda terms: [word for word in words 
                                                                if any(word.startswith(term) for term in terms)])
df_new['match_words2'] = df_new['two_grams'].apply(lambda terms: [word for word in words2 
                                                                if any(word.startswith(term) for term in terms)])
df_new['techincal skills'] =df_new['match_words']+df_new['match_words2']
                                                             
df_new.head()
  

,locations,date,title,description,company,salary,experience,skills,qualification,one_grams,two_grams,three_grams,Test_Flag,match_words,match_words2,techincal skills
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN,NaN,NaN,NaN,"[warehouse, data, streaming, etl, data, governance, business, intelligence, predictive, analytics, olap, data, science, framework, experience, experience, selling, data, analytics, software, solution, experience, global, analytics, market]","[warehouse data, data streaming, streaming etl, etl data, data governance, governance business, business intelligence, intelligence predictive, predictive analytics, analytics olap, olap data, data science, science framework, framework experience, experience experience, experience selling, selling data, data analytics, analytics software, software solution, solution experience, experience global, global analytics, analytics market]","[warehouse data streaming, data streaming etl, streaming etl data, etl data governance, data governance business, governance business intelligence, business intelligence predictive, intelligence predictive analytics, predictive analytics olap, analytics olap data, olap data science, data science framework, science framework experience, framework experience experience, experience experience selling, experience selling data, selling data analytics, data analytics software, analytics software solution, software solution experience, solution experience global, experience global analytics, global analytics market]",NaN,"[database, software]","[data analytics, data science, business intelligence, predictive analytics]","[database, software, data analytics, data science, business intelligence, predictive analytics]"
1,"Pretoria, Gauteng",2022-04-24 07:36:21+00:00,AWS Data Science Architect – Semi Remote – R742 Per hour,aws data science architect join dynamic team role responsible migration service general aws experience data science big data context docker kubernetes experience expert java developer,e-Merge IT Recruitment,R619 - 742 per hour,NaN,NaN,NaN,"[aws, data, science, architect, join, dynamic, team, role, responsible, migration, service, general, aws, experience, data, science, big, data, context, docker, kubernetes, experience, expert, java, developer]","[aws data, data science, science architect, architect join, join dynamic, dynamic team, team role, role responsible, responsible migration, migration service, service general, general aws, aws experience, experience data, data science, science big, big data, data context, context docker, docker kubernetes, kubernetes experience, experience expert, expert java, java developer]","[aws data science, data science architect, science architect join, architect join dynamic, join dynamic team, dynamic team role, team role responsible, role responsible migration, responsible migration service, migration service general, service general aws, general aws experience, aws experience data, experience data science, data science big, science big data, big data context, data context docker, context docker kubernetes, docker kubernetes experience, kubernetes experience expert, experience expert java, expert java developer]","[aws, developer, docker]","[java, aws, architect, developer, database, docker, kubernetes]","[data science, big data]","[java, aws, architect, developer, database, docker, kubernetes, data science, big data]"
2,"Johannesburg, Gauteng",2022-04-20 23:49:44+00:00,"Consultant, Data Science and Analytics",consultant data science analytics research technique software present finding peer within transunion manages integration data analysis,TransUnion,NaN,NaN,NaN,NaN,"[consultant, data, science, analytics, research, technique, software,

In [26]:
#Create a list of years of experience up to 5 years
#years_of_experience = ['1 year experience', '1 2 years experience', '1 3 years', '2 years', '2 3 years', '3 years', '3 4 years','4 years', '4 5 years','5 years', '3 5 years']

#df_new['three_grams'].isin(years_of_experience)

In [27]:
df_new.head(1)

,locations,date,title,description,company,salary,experience,skills,qualification,one_grams,two_grams,three_grams,Test_Flag
0,"Johannesburg, Gauteng",2022-04-29 07:52:13+00:00,Data Analytics Sales Specialist,warehouse data streaming etl data governance business intelligence predictive analytics olap data science framework experience experience selling data analytics software solution experience global analytics market,Google,NaN,NaN,NaN,NaN,"[warehouse, data, streaming, etl, data, governance, business, intelligence, predictive, analytics, olap, data, science, framework, experience, experience, selling, data, analytics, software, solution, experience, global, analytics, market]","[warehouse data, data streaming, streaming etl, etl data, data governance, governance business, business intelligence, intelligence predictive, predictive analytics, analytics olap, olap data, data science, science framework, framework experience, experience experience, experience selling, selling data, data analytics, analytics software, software solution, solution experience, experience global, global analytics, analytics market]","[warehouse data streaming, data streaming etl, streaming etl data, etl data governance, data governance business, governance business intelligence, business intelligence predictive, intelligence predictive analytics, predictive analytics olap, analytics olap data, olap data science, data science framework, science framework experience, framework experience experience, experience experience selling, experience selling data, selling data analytics, data analytics software, analytics software solution, software solution experience, solution experience global, experience global analytics, global analytics market]",NaN


In [28]:
# Define the function which takes a string and a list of words to extract as inputs
def listing_splitter(text, years_of_experience):
    # Try except to handle np.nans in input
    try:
        # Extract the list of flags
        flags = [l for l in years_of_experience if l in text.lower()]
        # If any flags were extracted then return the list
        if flags:
            return flags
        # Otherwise return np.nan
        else:
            return np.nan
    except AttributeError:
        return np.nan

# Apply the function to the column
df_new['Test_Flag'] = df_new['description'].apply(lambda x: listing_splitter(x, years_of_experience))
df_new.head(1)

NameError: ignored

In [ ]:
#Create a list of years of experience up to 5 years
years_of_experience = ['1 year', '1 2 years', '1 3 years', '2 years', '2 3 years', '3 years', '3 4 years','4 years', '4 5 years','5 years', '3 5 years']

def find_experience(description):
  """
  Function that searches for years of experience from the n grams created and 
  matches them with the list of experience created. 
  """

  a = [x for x in two_grams if x in years_of_experience]
  #b = [x for x in three_grams if x in years_of_experience]
  #a.append(b)
  return a#.append(b)

def add_experience(df_new):
  """
  A function that creates a new column called 'experience' then inserts the 
  experience required in years in the it.
  """
  df_new['experience'] = ''
  df_new['experience'] = df_new['description'].map(find_experience)
  return df_new['experience']

df_new['experience'].head()